In [1]:
import os
import sys

import re
import requests
from bs4 import BeautifulSoup

import sqlite3
import easyocr
import torch
import spacy

import scraping_v2_utils

sys.path.append(os.getcwd() + "/../../db")
sys.path.append(os.getcwd() + "/../../labos/nlp")

import sql_utils
import preprocessing
import model_utils

In [2]:
# TODO: Récup ces informations depuis la database
# Refs of catalog that dates already scrapped
catalog_dates_already_scraped = []  # refs
with open("./catalog_dates.txt") as catalog_dates_file:
    [
        catalog_dates_already_scraped.append(line.split("|")[0])
        for line in catalog_dates_file.read().split("\n")
    ]
print(catalog_dates_already_scraped)

# Refs of catalog with images already scrapped
catalog_images_already_scrapped = os.listdir("../../prod_dataset")
print(catalog_images_already_scrapped)

['24769797950543995-2', '24769797950543971-3', '24769797950543969-1', '24769797950543903-1', '24769797950544019-1', '24769797950544017-2', '24769797950543931-1', '24769797950544006-1', '']
['24769797950543903-1', '24769797950543995-2', '24769797950543969-1', '24769797950543971-3', '24769797950544019-1', '24769797950544006-1', '24769797950543931-1']


In [3]:
# Get the refs

url = "https://lapub.re/"
response = requests.get(url=url, timeout=(5,15))

soup = BeautifulSoup(response.text, "html.parser")

regex_pattern = re.compile(r"^https:\/\/lapub\.re\/imgpromo\/\d+.*maxi\.jpg$")
matching_imgs = soup.find_all("img", src=regex_pattern)

In [4]:
# Database

database_file_name = "db_prod_final.sqlite"

con = sqlite3.connect(os.getcwd() + f"/../../db/{database_file_name}")
cur = con.cursor()

sql_insert_catalog = sql_utils.get_sql_statement("insert_catalog_db_prod.sql")
sql_insert_page = sql_utils.get_sql_statement("insert_page_db_prod.sql")
sql_update_text = sql_utils.get_sql_statement("update_text_db_prod.sql")
sql_select_inference = sql_utils.get_sql_statement("select_pages_inference_v1.sql")
sql_select_image_without_text = sql_utils.get_sql_statement("select_path_of_image_without_text.sql")
sql_update_category = sql_utils.get_sql_statement("update_category_db_prod.sql") # To modify

## Insert catalog and page into db

In [18]:
# TODO: Clean and refactor
# TODO: Fix conditions => put on top if already "traité"
# Toute la pipeline se fait d'un coup !
for img in matching_imgs[:5]:
    catalog_ref = img["src"][26:-9]

    # * Check if dates not already scraped
    if catalog_ref not in catalog_dates_already_scraped:
        print("in")
        # Get dates
        (start_date, end_date) = scraping_v2_utils.get_dates(img.parent.get("href"))

        # Save in `catalog_dates.txt`
        scraping_v2_utils.save_dates(catalog_ref, start_date, end_date)

        # Save in db
        cur.execute(sql_insert_catalog, (catalog_ref, start_date, end_date))
    
        con.commit()
    

    # * Check if images not already downloaded
    if catalog_ref not in catalog_images_already_scrapped:
        print("in")
        # Get full links
        full_links = scraping_v2_utils.get_full_links(catalog_ref)
        print("full_links", full_links)
        # Download images
        scraping_v2_utils.download_all_images(full_links, catalog_ref)
        print("save in db")
        # Save in db
        for image_name in os.listdir(
            os.getcwd() + f"/../../prod_dataset/{catalog_ref}"
        ):
            # TODO: Verify not already in db ?
            cur.execute(
                sql_insert_page,
                (
                    f"./prod_dataset/{catalog_ref}/{image_name}",
                    catalog_ref,
                ),
            )
        con.commit()

in
in
2024-03-23 13:02:48.420437 => https://lapub.re/prospectus/24769797950544022-1/HTML/files/assets/common/page-html5-substrates/
catalog_dict {'24769797950544022-1': ['page0001_1.jpg', 'page0001_2.jpg', 'page0001_3.jpg', 'page0001_4.jpg', 'page0002_1.jpg', 'page0002_2.jpg', 'page0002_3.jpg', 'page0002_4.jpg', 'page0003_1.jpg', 'page0003_2.jpg', 'page0003_3.jpg', 'page0003_4.jpg', 'page0004_1.jpg', 'page0004_2.jpg', 'page0004_3.jpg', 'page0004_4.jpg', 'page0005_1.jpg', 'page0005_2.jpg', 'page0005_3.jpg', 'page0005_4.jpg', 'page0006_1.jpg', 'page0006_2.jpg', 'page0006_3.jpg', 'page0006_4.jpg', 'page0007_1.jpg', 'page0007_2.jpg', 'page0007_3.jpg', 'page0007_4.jpg', 'page0008_1.jpg', 'page0008_2.jpg', 'page0008_3.jpg', 'page0008_4.jpg', 'page0009_1.jpg', 'page0009_2.jpg', 'page0009_3.jpg', 'page0009_4.jpg', 'page0010_1.jpg', 'page0010_2.jpg', 'page0010_3.jpg', 'page0010_4.jpg', 'page0011_1.jpg', 'page0011_2.jpg', 'page0011_3.jpg', 'page0011_4.jpg', 'page0012_1.jpg', 'page0012_2.jpg', 'p

## Insert text into db

In [22]:
response = cur.execute(sql_select_image_without_text).fetchall()
print([elt[0][2:] for elt in response])

['prod_dataset/24769797950544022-1/page0001_4.jpg', 'prod_dataset/24769797950544022-1/page0023_4.jpg', 'prod_dataset/24769797950544022-1/page0004_4.jpg', 'prod_dataset/24769797950544022-1/page0021_4.jpg', 'prod_dataset/24769797950544022-1/page0020_4.jpg', 'prod_dataset/24769797950544022-1/page0015_4.jpg', 'prod_dataset/24769797950544022-1/page0018_4.jpg', 'prod_dataset/24769797950544022-1/page0003_4.jpg', 'prod_dataset/24769797950544022-1/page0022_4.jpg', 'prod_dataset/24769797950544022-1/page0012_4.jpg', 'prod_dataset/24769797950544022-1/page0019_4.jpg', 'prod_dataset/24769797950544022-1/page0009_4.jpg', 'prod_dataset/24769797950544022-1/page0006_4.jpg', 'prod_dataset/24769797950544022-1/page0011_4.jpg', 'prod_dataset/24769797950544022-1/page0017_4.jpg', 'prod_dataset/24769797950544022-1/page0010_4.jpg', 'prod_dataset/24769797950544022-1/page0025_4.jpg', 'prod_dataset/24769797950544022-1/page0007_4.jpg', 'prod_dataset/24769797950544022-1/page0002_4.jpg', 'prod_dataset/2476979795054402

In [23]:
# * Use OCR model to save raw text content in db
reader = easyocr.Reader(lang_list=["en"])

response = cur.execute(sql_select_image_without_text).fetchall()
page_missing_text_in_db = [elt[0] for elt in response]

for path in page_missing_text_in_db:
    ocr_output = reader.readtext(
        os.getcwd() + f"/../../{path[2:]}"
    )
    raw_text = "".join([elt[1] + " " for elt in ocr_output])

    print("Inserting text to db =>", path)
    print("raw text =>", raw_text)
    cur.execute(
        sql_update_text,
        (
            raw_text,
            path,
        ),
    )

    con.commit()

    # * Use NLP model to save insert page categories in db
    # response = cur.execute(sql_select_inference).fetchall()
    # """
    # Format: list[(id: int, text: str)]
    # """
    # vocabulary = preprocessing.get_vocabulary(os.getcwd() + "/../../labos/nlp/vocab_v1.txt")
    # labels = [i for i in range(16)]
    # model = model_utils.PageClassifier(len(vocabulary), len(labels))
    # model.load_state_dict(torch.load(os.getcwd() + "/../../labos/nlp/model_weight_v1.pth"))

    # nlp = spacy.load("fr_core_news_lg")

    # input = preprocessing.pipeline_from_raw_text_to_vectors(raw_text, nlp, vocabulary)
    # # TODO: Rename output
    # category = model_utils.predict(model, input)

    # cur.execute(sql_update_category, (category, response[0]))

    # con.commit()
# print("saved in db")

Inserting text to db => ./prod_dataset/24769797950544022-1/page0001_4.jpg
raw text => Carrefour Du lundi 18 mars Reunion aU dimanche 31 mars 2024 PAQUES CROYABLE ORIGINE NOUvELLE:ZELANDE Je 699 Gigot Vend8 Eeer avec 0s*** A rotir Entier Le kg Au royon Boucherie libre-service POUR VOTRE SANTE, PRATIQUEZ UNE ACTIVITE PHYSIQUE REGULIERE WWWMANGERBOUGERFR #CARREFOUR REUNION VOUS PROPOSE UN LARGE CHOIX DE PRODUITS DE MARQUES NATIONALES, DE PRODUITS DE MARQUE CARREFOUR ET DE PREMIERS PRIX A DES PRIX PARMI LES MOINS CHERS DU MARCHE choisis MOINS CHER 
Inserting text to db => ./prod_dataset/24769797950544022-1/page0023_4.jpg
raw text => 23 LOT 2 X 2 489 289 PRODUII 3 CNsaNo Nsto 75OML 0 DEODORANT Savons MONSAVON Dalmolie PALMOLIVE 3 OFFRE Sur une selection de parfums Natur La 48.90pme 8 Lot 2x200 ml: 12,22 € gr Le kg; 7,47 € € LOJI Dspogise en Lor Zx1oo ml Bgbompe = mble32 e doucbe 4 1 x2 le / : 24,45 € 89 Cadum c GEL Douche PaNIINE CADUM Gel Douche 3 299 PANTEM PINIENE Sur une selection DERMO

In [7]:
# TODO: Use AI model to add label into db_prod in another file ?!